# Analysing the redundancies of User Stories benefits

In [202]:
from load_data import load_datasets_with_annotations as loading
from dotenv import load_dotenv
from itertools import combinations
import pandas as pd
from openai import OpenAI
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_code = os.getenv('MODEL_VERSION')

In [203]:
client = OpenAI()
datasets: dict[str, list] = loading()

In [204]:
### This way of working with pandas is not recommend
### It has to be optimized to find a way to store for each text ther enteties and relations

def transform_data(data):
    rows = []
    row_enteties = []
    row_relations = []
    for line in data:
        for entity in line['entities']:
            entity_row = {
                "id": entity["id"],
                "label": entity["label"],
                "start_offset": entity["start_offset"],
                "end_offset": entity["end_offset"]
            }
            row_enteties.append(entity_row)
        for relation in line['relations']:
            relation_row = {
                "id": relation["id"],
                "type": relation["type"],
                "from id": relation["from_id"],
                "to id": relation["to_id"]
            }
            row_relations.append(relation_row)
        row = {
            'ID': line['id'],
            'User Story': line['text'],
            'Enteties': row_enteties,
            'Relations': row_relations
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [205]:
def transform_data_id_text(data: list):
    rows = []
    for entry in data:
        row = {
            'ID': entry['id'],
            'User Story': entry['text']
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [206]:
def transform_pairwise(df: pd.DataFrame):
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            'First ID': df.iloc[i, 0],
            'First User Story': df.iloc[i, 1],
            'Second ID': df.iloc[j, 0],
            'Second User Story': df.iloc[j, 1]
        }
        rows.append(row)
    return pd.DataFrame(rows)

## Request definition for User Stories

### Datatransformation to panda dataframes and numpy arrarys. 

### G02 federal funding redundancies

In [207]:
id_g02: str = "#G02#"
data_g02: list = datasets[id_g02]
df_g02: pd.DataFrame = transform_data_id_text(data_g02)
df_g02_pairs = transform_pairwise(df_g02)

In [208]:
display(df_g02)

,ID,User Story
0,215,"As a Data user, I want to have the 12-19-2017..."
1,216,"As a UI designer, I want to redesign the Reso..."
2,217,"As a UI designer, I want to report to the Age..."
3,218,"As a UI designer, I want to move on to round ..."
4,220,"As a UI designer, I want to move on to round ..."
...,...,...
90,308,"As a FABS user, I want to have my validations..."
91,309,"As a FABS user, I want to see correct status ..."
92,310,"As an agency user, I want to know when the su..."
93,311,"As an agency user, I want a landing page to n..."


In [209]:
display(df_g02_pairs)

,First ID,First User Story,Second ID,Second User Story
0,215,"As a Data user, I want to have the 12-19-2017...",216,"As a UI designer, I want to redesign the Reso..."
1,215,"As a Data user, I want to have the 12-19-2017...",217,"As a UI designer, I want to report to the Age..."
2,215,"As a Data user, I want to have the 12-19-2017...",218,"As a UI designer, I want to move on to round ..."
3,215,"As a Data user, I want to have the 12-19-2017...",220,"As a UI designer, I want to move on to round ..."
4,215,"As a Data user, I want to have the 12-19-2017...",221,"As a Developer , I want to be able to log bet..."
...,...,...,...,...
4460,309,"As a FABS user, I want to see correct status ...",311,"As an agency user, I want a landing page to n..."
4461,309,"As a FABS user, I want to see correct status ...",312,"As an agency user, I want to submit my data e..."
4462,310,"As an agency user, I want to know when the su...",311,"As an agency user, I want a landing page to n..."
4463,310,"As an agency user, I want to know when the su...",312,"As an agency user, I want to submit my data e..."


## Closing the client

In [210]:
client.close()